In [1]:
import pandas as pd
import os
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib.pyplot import cm

In [2]:
import sys
print(sys.path) 
sys.path.append("/cellar/users/mpagadal/Programs/anaconda3/lib/python3.7/site-packages")
sys.path.insert(1, '/cellar/users/mpagadal/Data/gcta/scripts')

['/data/cellardata2/users/mpagadal/projects/germline-immune/cell-type-specificity/notebooks', '/nrnb/opt/anaconda3.7/lib/python37.zip', '/nrnb/opt/anaconda3.7/lib/python3.7', '/nrnb/opt/anaconda3.7/lib/python3.7/lib-dynload', '', '/cellar/users/mpagadal/.local/lib/python3.7/site-packages', '/nrnb/opt/anaconda3.7/lib/python3.7/site-packages', '/nrnb/opt/anaconda3.7/lib/python3.7/site-packages/IPython/extensions', '/cellar/users/mpagadal/.ipython']


In [3]:
import networkx as nx
from ndex2.nice_cx_network import NiceCXNetwork 
import ndex2.client as nc
import ndex2

In [4]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [5]:
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
from matplotlib_venn import venn3

In [6]:
import statsmodels.stats.multitest as multi
from adjustText import adjust_text

In [7]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import json

## Get GTEx summary statistics for immune microenvironment SNPs from all tissue types

In [8]:
with open('/cellar/users/mpagadal/Data/resources/tcga/ensembl_map.json', 'r') as f:
    ensembl_dict = json.load(f)

In [9]:
ensembl_dict={k.split(".")[0]:v for k,v in ensembl_dict.items()}

In [10]:
def compile_gtex(directory,file_name,mapping):
    gtex_files=[x for x in os.listdir(directory)]
    total_gtex=pd.DataFrame()

    for x in gtex_files:
        cis=pd.read_csv(directory+x)
        cis["file"]=x.split(file_name)[0]
        total_gtex=total_gtex.append(cis)
    
    #format variant2 and gene
    total_gtex["variant2"]=total_gtex["variant_b37"].str.split("_").str[0]+":"+total_gtex["variant_b37"].str.split("_").str[1]+":"+total_gtex["variant_b37"].str.split("_").str[3]+":"+total_gtex["variant_b37"].str.split("_").str[2]
    #get gene ids for mapping
    total_gtex["gene_id"]=total_gtex["phenotype_id"].str.split(".").str[0]
    
    total_gtex["ref"]=total_gtex["variant_id"].str.split("_").str[2]
    total_gtex["alt"]=total_gtex["variant_id"].str.split("_").str[3]
    
    total_gtex_melt=pd.melt(total_gtex, id_vars=(["gene_id","slope","slope_se","ref","alt","file","pval_nominal","maf","tss_distance","variant_b37"]),value_vars=(["variant","variant2"]))
    
    total_gtex_melt["gene_name"]=total_gtex_melt["gene_id"].map(mapping)
    
    total_gtex_melt["assoc"]=total_gtex_melt["value"]+"_"+total_gtex_melt["gene_name"]
    
    return(total_gtex_melt)
    

In [11]:
gtex=compile_gtex("../data/rand_extracted_snps/",".snps.csv",ensembl_dict)

In [12]:
print(gtex.shape)
gtex.head()

(3406716, 14)


,gene_id,slope,slope_se,ref,alt,file,pval_nominal,maf,tss_distance,variant_b37,variable,value,gene_name,assoc
0,ENSG00000228463,-0.737765,0.396619,A,T,Brain_Cerebellum,0.064851,0.018617,917556,1_1150438_A_T_b37,variant,1:1150438:A:T,AP006222.2,1:1150438:A:T_AP006222.2
1,ENSG00000237094,0.930358,0.374408,A,T,Brain_Cerebellum,0.014073,0.018617,713441,1_1150438_A_T_b37,variant,1:1150438:A:T,RP4-669L17.10,1:1150438:A:T_RP4-669L17.10
2,ENSG00000237094,0.852446,0.449281,C,T,Brain_Cerebellum,0.059729,0.013298,833254,1_1270251_C_T_b37,variant,1:1270251:C:T,RP4-669L17.10,1:1270251:C:T_RP4-669L17.10
3,ENSG00000237094,0.063429,0.142695,A,G,Brain_Cerebellum,0.657328,0.159574,872888,1_1309885_A_G_b37,variant,1:1309885:A:G,RP4-669L17.10,1:1309885:A:G_RP4-669L17.10
4,ENSG00000225972,-0.016371,0.250645,A,T,Brain_Cerebellum,0.948010,0.018617,585996,1_1150438_A_T_b37,variant,1:1150438:A:T,MTND1P23,1:1150438:A:T_MTND1P23


## Combine gtex information with TCGA association information

In [14]:
#get randomized genes
snps=pd.read_csv("../data/gtex_rand.csv",delimiter="\t")
snps["gene_name"]=snps["gene_id"].map(ensembl_dict)
snps["assoc"]=snps["variant"]+"_"+snps["gene_name"]

In [15]:
snps_gtex=pd.merge(snps[["assoc"]],gtex,on="assoc",how="left")

In [16]:
snps_gtex["assoc"]=snps_gtex["assoc"].str.replace("-",".")

In [17]:
snps_gtex["assoc2"]=snps_gtex["value"].str.rsplit(":",2).str[0]+":"+snps_gtex["value"].str.split(":").str[3]+":"+snps_gtex["value"].str.split(":").str[2]+"_"+snps_gtex["gene_name"]

In [18]:
snps_gtex.columns

Index(['assoc', 'gene_id', 'slope', 'slope_se', 'ref', 'alt', 'file',
       'pval_nominal', 'maf', 'tss_distance', 'variant_b37', 'variable',
       'value', 'gene_name', 'assoc2'],
      dtype='object')

In [19]:
snps_gtex=pd.melt(snps_gtex,id_vars=(["variant_b37","tss_distance","slope","slope_se","ref","alt","file"]),value_vars=(["assoc","assoc2"]))

In [20]:
del snps_gtex["variable"]

In [21]:
snps_gtex=snps_gtex.rename(columns={"value":"assoc"})

In [22]:
len(snps_gtex["assoc"].unique())

2000

## Identify of GTEx snps that were found that are cis associations

## See concordance between associations

In [23]:
tpm_cancer=pd.read_csv("../data/tpm.rand.gtex.tsv",delimiter="\t")
firebrowse_cancer=pd.read_csv("../data/firebrowse.rand.gtex.tsv",delimiter="\t")
pancanatlas_cancer=pd.read_csv("../data/pancanatlas.rand.gtex.tsv",delimiter="\t")

In [24]:
#get tissue mappings
tissue_mapping=pd.read_csv("../data/gtex.tcga.tissue.csv")
tissue_mapping["combo"]=tissue_mapping["gtex tissues"]+"."+tissue_mapping["TCGA"]

In [25]:
tissue_mapping["whole_blood combo"]="Whole_Blood"+"."+tissue_mapping["TCGA"]

In [26]:
def make_bootstrap(df,name,combos):
    df["assoc"]=df["ID"]+"_"+df["pheno"]
    compare_beta=pd.merge(snps_gtex,df,on="assoc")
    
    compare_beta["combo"]=compare_beta["file"]+"."+compare_beta["file_name"]
    
    if combos == "regular":
        compare_beta=compare_beta[compare_beta["combo"].isin(tissue_mapping["combo"].tolist())]
    if combos == "blood":
        compare_beta=compare_beta[compare_beta["combo"].isin(tissue_mapping["whole_blood combo"].tolist())]
    
    for x in compare_beta["combo"].unique():
        compare_beta_combo=compare_beta[compare_beta["combo"]==x]
        #reverse slope for cases of ref/alt mismatch
        compare_beta_combo["slope"]=np.where(compare_beta_combo["alt"] != compare_beta_combo["ALT"], (compare_beta_combo["slope"]*-1),compare_beta_combo["slope"])
        print(x)
        print(compare_beta_combo.shape)
        compare_beta_combo["z"]=(compare_beta_combo["BETA"]-compare_beta_combo["slope"])/compare_beta_combo["slope_se"]
        compare_beta_combo.to_csv(name+"."+x+".csv")
    

In [27]:
make_bootstrap(tpm_cancer,"../for_bootstrapping/tpm/tpm.rand","regular")
make_bootstrap(firebrowse_cancer,"../for_bootstrapping/firebrowse/firebrowse.rand","regular")
make_bootstrap(pancanatlas_cancer,"../for_bootstrapping/pancanatlas/pancanatlas.rand","regular")

/nrnb/opt/anaconda3.7/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/nrnb/opt/anaconda3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Thyroid.THCA
(1006, 23)
Brain_Substantia_nigra.GBM
(988, 23)
Brain_Substantia_nigra.LGG
(988, 23)
Brain_Anterior_cingulate_cortex_BA24.GBM
(986, 23)
Brain_Anterior_cingulate_cortex_BA24.LGG
(986, 23)
Brain_Spinal_cord_cervical_c-1.GBM
(994, 23)
Brain_Spinal_cord_cervical_c-1.LGG
(994, 23)
Brain_Cortex.GBM
(987, 23)
Brain_Cortex.LGG
(987, 23)
Prostate.PRAD
(1010, 23)
Small_Intestine_Terminal_Ileum.COAD
(1011, 23)
Brain_Amygdala.GBM
(983, 23)
Brain_Amygdala.LGG
(983, 23)
Lung.LUAD
(1014, 23)
Lung.LUSC
(1015, 23)
Testis.TGCT
(1025, 23)
Liver.LIHC
(953, 23)
Brain_Hippocampus.GBM
(986, 23)
Brain_Hippocampus.LGG
(986, 23)
Brain_Hypothalamus.GBM
(997, 23)
Brain_Hypothalamus.LGG
(997, 23)
Pituitary.GBM
(1007, 23)
Pituitary.LGG
(1007, 23)
Breast_Mammary_Tissue.BRCA
(1028, 23)
Brain_Caudate_basal_ganglia.GBM
(989, 23)
Brain_Caudate_basal_ganglia.LGG
(989, 23)
Brain_Frontal_Cortex_BA9.GBM
(989, 23)
Brain_Frontal_Cortex_BA9.LGG
(989, 23)
Kidney_Cortex.KICH
(993, 23)
Kidney_Cortex.KIRC
(994, 23)
Ki

In [28]:
make_bootstrap(tpm_cancer,"../for_bootstrapping/tpm/tpm.rand","blood")
make_bootstrap(firebrowse_cancer,"../for_bootstrapping/firebrowse/firebrowse.rand","blood")
make_bootstrap(pancanatlas_cancer,"../for_bootstrapping/pancanatlas/pancanatlas.rand","blood")

/nrnb/opt/anaconda3.7/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/nrnb/opt/anaconda3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Whole_Blood.ACC
(892, 23)
Whole_Blood.BRCA
(892, 23)
Whole_Blood.COAD
(892, 23)
Whole_Blood.ESCA
(891, 23)
Whole_Blood.GBM
(892, 23)
Whole_Blood.HNSC
(890, 23)
Whole_Blood.KICH
(891, 23)
Whole_Blood.KIRC
(892, 23)
Whole_Blood.KIRP
(892, 23)
Whole_Blood.LGG
(892, 23)
Whole_Blood.LIHC
(891, 23)
Whole_Blood.LUAD
(891, 23)
Whole_Blood.LUSC
(892, 23)
Whole_Blood.OV
(892, 23)
Whole_Blood.PAAD
(892, 23)
Whole_Blood.PRAD
(891, 23)
Whole_Blood.SKCM
(887, 23)
Whole_Blood.STAD
(892, 23)
Whole_Blood.TGCT
(892, 23)
Whole_Blood.THCA
(892, 23)
Whole_Blood.UCEC
(891, 23)
Whole_Blood.UCS
(892, 23)
Whole_Blood.ACC
(892, 23)
Whole_Blood.BRCA
(892, 23)
Whole_Blood.COAD
(892, 23)
Whole_Blood.ESCA
(891, 23)
Whole_Blood.GBM
(892, 23)
Whole_Blood.HNSC
(890, 23)
Whole_Blood.KICH
(891, 23)
Whole_Blood.KIRC
(892, 23)
Whole_Blood.KIRP
(892, 23)
Whole_Blood.LGG
(892, 23)
Whole_Blood.LIHC
(891, 23)
Whole_Blood.LUAD
(891, 23)
Whole_Blood.LUSC
(892, 23)
Whole_Blood.OV
(892, 23)
Whole_Blood.PAAD
(892, 23)
Whole_Blood.

In [30]:
files=[x.split("tpm.")[1] for x in os.listdir("../for_bootstrapping/tpm/")]

In [31]:
files=[x.replace("rand.","") for x in files]
files=[x.replace("test.","") for x in files]

In [32]:
len([x for x in set(files)])

75

In [33]:
direct=[]

for i in range(75):
    direct.append("tpm")
for i in range(75):
    direct.append("firebrowse")
for i in range(75):
    direct.append("pancanatlas")

In [34]:
len(direct)

225

In [35]:
" ".join(direct)

'tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm tpm firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebrowse firebr